在运行受输入/输出（IO）限制的代码（例如，并发地向聊天模型提供商发起 API 请求）时，使用异步编程范式可以带来显著的性能提升。
要将图的同步实现转换为异步实现，需要有以下步骤：
1. 更新节点定义：将节点函数定义中的 def 改为 async def。
2. 更新内部代码：适当添加 await 关键字。
3. 调用图：根据需要使用 .ainvoke 或 .astream 来调用图。
由于许多 LangChain 对象都实现了 Runnable Protocol（可运行协议），该协议包含了所有同步方法的异步变体，因此通常可以非常快速地将同步图升级为异步图。

In [ ]:
from langchain.chat_models import init_chat_model
from langgraph.graph import MessagesState, StateGraph

async def node(state: MessagesState):  
    new_message = await llm.ainvoke(state["messages"])  
    return {"messages": [new_message]}

builder = StateGraph(MessagesState).add_node(node).set_entry_point("node")
graph = builder.compile()

input_message = {"role": "user", "content": "Hello"}
result = await graph.ainvoke({"messages": [input_message]})